In [252]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from scipy import stats
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,KFold
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso,LassoCV,RidgeCV,ElasticNet,ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,GradientBoostingRegressor,StackingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.pipeline import make_pipeline
from math import sqrt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [194]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submission=pd.read_csv("sample_submission.csv")
ntrain=train.shape[0]
ntest=test.shape[0]
#train_ID = train['Id']
#test_ID = test['Id']
#train.drop("Id", axis = 1, inplace = True)
#test.drop("Id", axis = 1, inplace = True)
y=train.SalePrice
#y=np.log1p(y)
data=pd.concat([train,test],axis=0,sort=False)
data.drop(['SalePrice',"GarageYrBlt"], axis = 1, inplace = True)



In [195]:
pd.set_option('display.max_rows', 5000)
train_test_null_info=pd.DataFrame(data.isnull().sum(),columns=['Count of NaN'])
train_test_dtype_info=pd.DataFrame(data.dtypes,columns=['DataTypes'])
train_tes_info=pd.concat([train_test_null_info,train_test_dtype_info],axis=1)
train_tes_info

,Count of NaN,DataTypes
Id,0,int64
MSSubClass,0,int64
MSZoning,4,object
LotFrontage,486,float64
LotArea,0,int64
Street,0,object
Alley,2721,object
LotShape,0,object
LandContour,0,object
Utilities,2,object


In [196]:
data.loc[data['Fireplaces']==0,'FireplaceQu']='Nothing'
data['LotFrontage'] = data['LotFrontage'].fillna(data.groupby('1stFlrSF')['LotFrontage'].transform('mean'))
data['LotFrontage'].interpolate(method='linear',inplace=True)
data['LotFrontage']=data['LotFrontage'].astype(int)
data['MasVnrArea'] = data['MasVnrArea'].fillna(data.groupby('MasVnrType')['MasVnrArea'].transform('mean'))
data['MasVnrArea'].interpolate(method='linear',inplace=True)
data['MasVnrArea']=data['MasVnrArea'].astype(int)
data["Fence"] = data["Fence"].fillna("None")
data["FireplaceQu"] = data["FireplaceQu"].fillna("None")
data["Alley"] = data["Alley"].fillna("None")
data["PoolQC"] = data["PoolQC"].fillna("None")
data["MiscFeature"] = data["MiscFeature"].fillna("None")
data.loc[data['BsmtFinSF1']==0,'BsmtFinType1']='Unf'
data.loc[data['BsmtFinSF2']==0,'BsmtQual']='TA'
data['Total_SF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']      
data['YrBltRmd']=data['YearBuilt']+data['YearRemodAdd']
data['Total_Square_Feet'] = (data['BsmtFinSF1'] + data['BsmtFinSF2'] + data['1stFlrSF'] + data['2ndFlrSF'])
data['Total_Bath'] = (data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath']))
data['Total_Porch_Area'] = (data['OpenPorchSF'] + data['3SsnPorch'] + data['EnclosedPorch'] + data['ScreenPorch'] + data['WoodDeckSF'])
data['existpool'] = data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
data['existfireplace'] = data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
data['old_house'] = data['YearBuilt'].apply(lambda x: 1 if x <1995 else 0)

for i in data.columns:
    if 'SalePrice' not in i:
        if 'object' in str(data[str(i)].dtype):
            data[str(i)]=data[str(i)].fillna(method='ffill')
            
            

In [197]:
columns = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 'YrSold', 'MoSold',
           'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope', 'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond')

for col in columns:
    lbl_enc = LabelEncoder() 
    lbl_enc.fit(list(data[col].values)) 
    data[col] = lbl_enc.transform(list(data[col].values))

In [198]:
data=pd.get_dummies(data)


In [199]:
train=data[0:1460]
test=data[1460:2919]
train["SalePrice"]=y

In [202]:
train.interpolate(method='linear',inplace=True)
test.interpolate(method='linear',inplace=True)

In [204]:
train = train.drop(train[(train.OverallQual==4) & (train.SalePrice>200000)].index)
train = train.drop(train[(train.OverallQual==10) & (train.SalePrice<200000)].index)
train = train.drop(train[(train.Total_SF>=10000) & (train.SalePrice<200000)].index)
train = train.drop(train[(train.GrLivArea>5000) & (train.SalePrice<200000)].index)
train = train.drop(train[(train.GrLivArea<3000) & (train.SalePrice>550000)].index)
train = train.drop(train[(train.Total_Bath>4) & (train.SalePrice<150000)].index)
train = train.drop(train[(train.Total_Bath<5) & (train.SalePrice>550000)].index)
train = train.drop(train[(train.GarageArea>1200) & (train.SalePrice<100000)].index)
train = train.drop(train[(train.GarageArea<1000) & (train.SalePrice>500000)].index)
train = train.drop(train[(train.TotalBsmtSF>6000) & (train.SalePrice<200000)].index)
train = train.drop(train[(train.FullBath<1) & (train.SalePrice>250000)].index)
train = train.drop(train[(train.TotRmsAbvGrd==14) & (train.SalePrice<300000)].index)
train = train.drop(train[(train.TotRmsAbvGrd==10) & (train.SalePrice>700000)].index)
train = train.drop(train[(train.YearBuilt<1950) & (train.SalePrice>350000)].index)
train = train.drop(train[(train.YearRemodAdd<1970) & (train.SalePrice>350000)].index)
train = train.drop(train[(train.Foundation_PConc==0) & (train.SalePrice>450000)].index)
train = train.drop(train[(train.MasVnrArea>=1500) & (train.SalePrice<250000)].index)
train = train.drop(train[(train.MasVnrArea<=750) & (train.SalePrice>550000)].index)
train = train.drop(train[(train.HeatingQC.isin([2,4])) & (train.SalePrice>350000)].index)
train = train.drop(train[(train.GarageFinish.isin([1,2])) & (train.SalePrice>470000)].index)
train = train.drop(train[(train.old_house==0) & (train.SalePrice<100000)].index)
train = train.drop(train[(train.old_house==1) & (train.SalePrice>450000)].index)
train = train.drop(train[(train.KitchenQual==2) & (train.SalePrice>600000)].index)
train = train.drop(train[(train.KitchenQual==3) & (train.SalePrice>350000)].index)
train = train.drop(train[(train.ExterQual==2) & (train.SalePrice>500000)].index)
train = train.drop(train[(train.BsmtFullBath==0) & (train.SalePrice>650000)].index)

train = train[train.GarageArea * train.GarageCars < 3700]
train = train[(train.FullBath + (train.HalfBath*0.5) + train.BsmtFullBath + (train.BsmtHalfBath*0.5))<5]

In [205]:
train["SalePrice"]=np.log1p(train["SalePrice"])

In [206]:
X=train.drop(['SalePrice'],axis=1)
y=train.SalePrice

In [207]:
std_scaler=StandardScaler()
robust_scaler=RobustScaler()
power_scaler=PowerTransformer()

X_std=std_scaler.fit_transform(X)
X_rbst=robust_scaler.fit_transform(X)
X_pwr=power_scaler.fit_transform(X)

test_std=std_scaler.fit_transform(test)
test_rbst=robust_scaler.fit_transform(test)
test_pwr=power_scaler.fit_transform(test)

In [214]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pwr, y,test_size = 0.3, random_state = 5)

In [232]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.03, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       ).fit(X_pwr,y)
lgbm_pred=lightgbm.predict(X_test)
print("LGBM Regressor R^2 score:",r2_score(y_test,lgbm_pred))
print('LGBM Regressor Mean Absolute Error Accuracy: ',mean_absolute_error(y_test,lgbm_pred))
print('LGBM Regressor Mean Squared Error Accuracy: ',mean_squared_error(y_test,lgbm_pred))


Gradient Boosting Regressor R^2 score: 0.9871270683564197
Gradient Boosting Regressor Mean Absolute Error Accuracy:  0.03288155062929827
Gradient Boosting Regressor Mean Squared Error Accuracy:  0.0018584579837496325


In [233]:
lambdas=[1e-9,1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.1, 1, 10]
l1ratio =[0.8, 0.83, 0.85, 0.87, 0.9,0.92, 0.95,0.97, 0.99, 1]
kfolds = KFold(n_splits=8, shuffle=True, random_state=42)

ridge_cv=make_pipeline(RidgeCV(alphas=lambdas,cv=kfolds))
ridge_model=ridge_cv.fit(X_pwr,y)
ridge_pred=ridge_model.predict(X_test)
print("Ridge Regression R^2 score:",r2_score(y_test,ridge_pred))
print('Ridge Regression Mean Absolute Error Accuracy: ',mean_absolute_error(y_test,ridge_pred))
print('Ridge Regression Mean Squared Error Accuracy: ',mean_squared_error(y_test,ridge_pred))

Ridge Regression R^2 score: 0.9454518409472935
Ridge Regression Mean Absolute Error Accuracy:  0.06684861111134105
Ridge Regression Mean Squared Error Accuracy:  0.007875087392459137


In [235]:
lasso = make_pipeline(LassoCV(max_iter=1e7,alphas=lambdas,cv=kfolds))
lasso_model=lasso.fit(X_pwr,y)
lasso_pred=lasso_model.predict(X_test)
print("\nLasso Regression R^2 score:",r2_score(y_test,lasso_pred))
print('Lasso Regression Mean Absolute Error Accuracy: ',mean_absolute_error(y_test,lasso_pred))
print('Lasso Regression Mean Squared Error Accuracy:',mean_squared_error(y_test,lasso_pred))



Lasso Regression R^2 score: 0.9445641914152921
Lasso Regression Mean Absolute Error Accuracy:  0.06693975041690309
Lasso Regression Mean Squared Error Accuracy: 0.008003236861841456


In [248]:
gbr = GradientBoostingRegressor(n_estimators=1500, learning_rate=0.01012, max_depth=6, max_features='sqrt', min_samples_leaf=15, min_samples_split=14, loss='huber', random_state =42)
gbr.fit(X_pwr, y)
gbr_pred=gbr.predict(X_test)
print('-'*10+'GBR'+'-'*10)
print('R square Accuracy: ',r2_score(y_test,gbr_pred))
print('Mean Absolute Error Accuracy: ',mean_absolute_error(y_test,gbr_pred))
print('Mean Squared Error Accuracy: ',mean_squared_error(y_test,gbr_pred))

----------GBR----------
R square Accuracy:  0.9686402576639701
Mean Absolute Error Accuracy:  0.03795340383342163
Mean Squared Error Accuracy:  0.004527388564343919


In [250]:
xgb = XGBRegressor()

xgb_model=XGBRegressor(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 seed=27).fit(X_pwr,y)

xgb_pred=xgb_model.predict(X_test)
print("XGB Regressor R^2 score:",r2_score(y_test,xgb_pred))
print('XGB Regressor Mean Absolute Error Accuracy: ',mean_absolute_error(y_test,xgb_pred))
print('XGB Regressor Mean Squared Error Accuracy: ',mean_squared_error(y_test,xgb_pred))



XGB Regressor R^2 score: 0.9999963957827585
XGB Regressor Mean Absolute Error Accuracy:  0.0005233481077169616
XGB Regressor Mean Squared Error Accuracy:  5.203388391321685e-07


In [253]:
estimators = [('lgbm', lightgbm),
              ('gbr', gbr),   
              ('XGB', xgb_model)   
             ]

stack_reg=StackingRegressor(estimators=estimators,final_estimator=xgb_model,n_jobs=-1)
stack_reg.fit(X_pwr, y)
y_head=stack_reg.predict(X_test)
print('-'*10+'StackingRegressor'+'-'*10)
print('R square Accuracy: ',r2_score(y_test,y_head))
print('Mean Absolute Error Accuracy: ',mean_absolute_error(y_test,y_head))
print('Mean Squared Error Accuracy: ',mean_squared_error(y_test,y_head))

----------StackingRegressor----------
R square Accuracy:  0.959062833476752
Mean Absolute Error Accuracy:  0.05459235880817699
Mean Squared Error Accuracy:  0.005910075968993401
